In [83]:
import dicom2nifti
import nibabel as nib
import nilearn as nil
import scipy.ndimage as ndi
import matplotlib.pyplot as plt
import os
import glob
from pathlib import Path
import SimpleITK as sitk # para calcular rangos
import numpy as np
import io
from PIL import Image
import random
import sys
import pandas as pd
import math
import pandas as pd
import sklearn
import sklearn.metrics
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

In [84]:
sys.path.append("/Users/hugom/PET-IA/src")

In [85]:
from datasets import ADNIDataset, FleniMyriamDataset, BaseDataset
from transforms import TransformGridImage, ToLabelOutputConfigurable
from train_lib import train_model, set_parameter_requires_grad, initialize_model
from util import collectAllData
import torch
import torchvision
from torchvision import transforms, utils, models, datasets

In [77]:
config = {
  "last_3_classes": {
    "weights": "/Users/hugom/Tesis/ExperimentosServer/muestraFull3700_9/muestraFull3700_9_dxlast_3classes_0_epoch21.pth",
    "num_classes": 3
  },
  "sev_3_classes": {
    "weights": "/Users/hugom/Tesis/ExperimentosServer/muestraFull3700_9/muestraFull3700_9_dxmost_severe_3classes_0_epoch31.pth",
    "num_classes": 3
  },
  "visit_3_classes": {
    "weights": "/Users/hugom/Tesis/ExperimentosServer/muestraFull3700_9/muestraFull3700_9_dxvisit_3classes_0_epoch10.pth",
    "num_classes": 3
  },
  "last_2classes": {
    "weights": "/Users/hugom/Tesis/ExperimentosServer/muestraFull3700_9/muestraFull3700_9_dxlast_2classes_0_epoch18.pth",
    "num_classes": 2
  },
  "sev_2classes": {
    "weights": "/Users/hugom/Tesis/ExperimentosServer/muestraFull3700_9/muestraFull3700_9_dxmost_severe_2classes_0_epoch16.pth",
    "num_classes": 2
  },
  "visit_2classes": {
    "weights": "/Users/hugom/Tesis/ExperimentosServer/muestraFull3700_9/muestraFull3700_9_dxvisit_2classes_0_epoch5.pth",
    "num_classes": 2
  }
}


In [87]:
def executeModel(dataloader, weights, num_classes):
    featureExtract = False
    dropoutRate = 0.6
    auxEnabled = True
    usePretrained = False
    model_ft, input_size = initialize_model('inception', num_classes, featureExtract, dropoutRate, auxEnabled, use_pretrained=usePretrained)
    device = torch.device(deviceName if torch.cuda.is_available() else "cpu")
    model_state_dict = torch.load(weights, map_location=device)
    model_ft.load_state_dict(model_state_dict)

    model_ft.to(device)
    
    testY, predY = collectAllData(dataloader, model_ft, device, num_classes)
    
    _, preds = torch.max(torch.from_numpy(predY), 1)
    preds = preds.cpu().numpy()
    
    acc = sklearn.metrics.accuracy_score(testY, preds)
    bacc = sklearn.metrics.balanced_accuracy_score(testY, preds)
    
    print(testY)
    print(preds)
    
    print(f"Accuracy: {acc}")
    print(f"B acc: {bacc}")
    matrix = sklearn.metrics.confusion_matrix(testY, preds)
    
    print("Confusion matrix:")
    print(matrix)
    
    if (num_classes == 2):
        lr_auc = roc_auc_score(testY, torch.from_numpy(predY[:, 1]), average = None)
        print(f"AUC ROC: {lr_auc}")
        return
    
    print("------------")
    print("Merging 2 with 0")
    
    predY[:, 0] = predY[:, 0] + predY[:, 2]
    predY = predY[:, 0:2]
    
    _, preds = torch.max(torch.from_numpy(predY), 1)
    preds = preds.cpu().numpy()
    
    acc = sklearn.metrics.accuracy_score(testY, preds)
    bacc = sklearn.metrics.balanced_accuracy_score(testY, preds)
    
    print(f"Accuracy merged: {acc}")
    print(f"B acc merged: {bacc}")
    
    print("Confusion matrix merged:")
    matrix = sklearn.metrics.confusion_matrix(testY, preds)
    print(matrix)
    
    lr_auc = roc_auc_score(testY, torch.from_numpy(predY[:, 1]), average = None)
    print(f"AUC ROC: {lr_auc}")

In [81]:
csvFile = '/Users/hugom/PET-IA/Sets/fleni-PET_clasificados60.csv'
imagesFolder = '/Users/hugom/Tesis/Imagenes/fleni-stripped-preprocessed4'
#mean = 3364.6066073463076 # Uso el de myriam db
#std = 7271.672596534478   # Uso el de myriam db
mean = 3864.730224609375
std = 8282.332521699427
means = [mean, mean, mean]
stds = [std, std, std]
ebrainsTransforms = torchvision.transforms.Compose([
    TransformGridImage(),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(means, stds)
])
dicti = {
    "AD": 1,
    "non-AD": 0
}
dataset = BaseDataset('fleni60', csvFile, imagesFolder, studyIDLabel = 'anon_id', transform = ebrainsTransforms, target_transform = ToLabelOutputConfigurable(dicti), truthLabel = 'Conclusion PET')
dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False, num_workers=0)

In [82]:
for key in config:
    print(f"Ejecutando {key}")
    executeModel(dataloader, **config[key])
    print("#################")

Ejecutando last_3_classes


/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


num featurs2048
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1 0 0 1 1 1 1 1 1 0 1 1 0 1 0 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 0 1 0 0 1 0
 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1]
Accuracy: 0.5
B acc: 0.4065934065934066
Confusion matrix:
[[ 2 19]
 [11 28]]
------------
Merging 2 with 0
Accuracy merged: 0.5166666666666667
B acc merged: 0.43040293040293043
Confusion matrix merged:
[[ 3 18]
 [11 28]]
AUC ROC: 0.4694749694749695
#################
Ejecutando sev_3_classes


/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


num featurs2048


...

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1 2 2 2 1 2 2 1 1 2 2 2 1 0 2 2 2 2 2 2 2 0 2 2 2 1 2 2 2 2 2 1 2 2 2 2 2
 2 2 2 2 2 1 2 2 2 2 2 2 2 2 1 2 2 2 2 2 1 2 2]
Accuracy: 0.11666666666666667
B acc: 0.08974358974358974
Confusion matrix:
[[ 0  3 18]
 [ 2  7 30]
 [ 0  0  0]]
------------
Merging 2 with 0
Accuracy merged: 0.4
B acc merged: 0.5274725274725275
Confusion matrix merged:
[[20  1]
 [35  4]]
AUC ROC: 0.5543345543345544
#################
Ejecutando last_2classes
num featurs2048
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Accuracy: 0.35
B acc: 0.5
Confusion matrix:
[[21  0]
 [39  

/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


num featurs2048
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1 0 0 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0
 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1]
Accuracy: 0.5666666666666667
B acc: 0.46886446886446886
Confusion matrix:
[[ 3 18]
 [ 8 31]]
AUC ROC: 0.4725274725274725
#################
Ejecutando visit_2classes


/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


num featurs2048
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Accuracy: 0.35
B acc: 0.5
Confusion matrix:
[[21  0]
 [39  0]]
AUC ROC: 0.4444444444444445
#################


# Nuevos

In [91]:
configNew = {
  "last_cn_vs_ad": {
    "weights": "/Users/hugom/Tesis/ExperimentosServer/muestraFull3700_8/muestraFull3700_8_dxlast_2classes_CN,AD_0_epoch42.pth",
    "num_classes": 2
  },
  "sev_cn_vs_ad": {
    "weights": "/Users/hugom/Tesis/ExperimentosServer/muestraFull3700_8/muestraFull3700_8_dxmost_severe_2classes_CN,AD_0_epoch12.pth",
    "num_classes": 2
  },
  "visit_cn_vs_ad": {
    "weights": "/Users/hugom/Tesis/ExperimentosServer/muestraFull3700_8/muestraFull3700_8_dxvisit_2classes_CN,AD_0_epoch19.pth",
    "num_classes": 2
  },
  "last_mci_vs_ad": {
    "weights": "/Users/hugom/Tesis/ExperimentosServer/muestraFull3700_8/muestraFull3700_8_dxlast_2classes_MCI,AD_0_epoch45.pth",
    "num_classes": 2
  },
  "sev_mci_vs_ad": {
    "weights": "/Users/hugom/Tesis/ExperimentosServer/muestraFull3700_8/muestraFull3700_8_dxmost_severe_2classes_MCI,AD_0_epoch17.pth",
    "num_classes": 2
  },
  "visit_mci_vs_ad": {
    "weights": "/Users/hugom/Tesis/ExperimentosServer/muestraFull3700_8/muestraFull3700_8_dxvisit_2classes_MCI,AD_0_epoch1.pth",
    "num_classes": 2
  }
}


In [89]:
csvFile = '/Users/hugom/PET-IA/Sets/fleni-PET_clasificados60.csv'
imagesFolder = '/Users/hugom/Tesis/Imagenes/fleni-stripped-preprocessed4'
#mean = 3364.6066073463076 # Uso el de myriam db
#std = 7271.672596534478   # Uso el de myriam db
mean = 3864.730224609375
std = 8282.332521699427
means = [mean, mean, mean]
stds = [std, std, std]
ebrainsTransforms = torchvision.transforms.Compose([
    TransformGridImage(),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(means, stds)
])
dicti = {
    "AD": 1,
    "non-AD": 0
}
dataset = BaseDataset('fleni60', csvFile, imagesFolder, studyIDLabel = 'anon_id', transform = ebrainsTransforms, target_transform = ToLabelOutputConfigurable(dicti), truthLabel = 'Conclusion PET')
dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False, num_workers=0)

In [92]:
for key in configNew:
    print(f"Ejecutando {key}")
    executeModel(dataloader, **configNew[key])
    print("#################")

Ejecutando last_cn_vs_ad


/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


num featurs2048
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 0 1 1 0 1 1 1 0 0 1 0 0 0 0 0 1 0 1 1 1 1
 0 0 0 0 1 1 0 0 0 0 1 0 1 0 1 0 1 1 1 1 1 1 0]
Accuracy: 0.5833333333333334
B acc: 0.5586080586080586
Confusion matrix:
[[10 11]
 [14 25]]
AUC ROC: 0.6300366300366301
#################
Ejecutando sev_cn_vs_ad


/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


num featurs2048
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Accuracy: 0.65
B acc: 0.5
Confusion matrix:
[[ 0 21]
 [ 0 39]]
AUC ROC: 0.4761904761904762
#################
Ejecutando visit_cn_vs_ad


...

num featurs2048
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0 1 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0]
Accuracy: 0.4166666666666667
B acc: 0.5183150183150182
Confusion matrix:
[[18  3]
 [32  7]]
AUC ROC: 0.5885225885225885
#################
Ejecutando sev_mci_vs_ad


/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


num featurs2048
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0
 1 0 1 1 0 0 1 1 1 1 0 0 0 0 0 1 0 0 0 0 0 0 1]
Accuracy: 0.35
B acc: 0.41208791208791207
Confusion matrix:
[[13  8]
 [31  8]]
AUC ROC: 0.42002442002442003
#################
Ejecutando visit_mci_vs_ad


/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/Users/hugom/Library/Python/3.9/lib/python/site-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


num featurs2048
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Accuracy: 0.35
B acc: 0.5
Confusion matrix:
[[21  0]
 [39  0]]
AUC ROC: 0.6886446886446886
#################
